## Run UDFs on Merlin live streams

If you want to use this with the simulated data source, run something like this in the background:

`libertem-live-mib-sim ~/Data/default.hdr --cached=MEM`

On Linux, `MEMFD` is also supported as a cache. Use `NONE` to deactivate the cache.

* Make sure to adjust the `SCAN_SIZE` below to match the scan of the data source!
* This notebook requires the `bqplot` extra of LiberTEM: `pip install libertem[bqplot]`

In [1]:
%load_ext autoreload

In [2]:
# Uncomment to use Matplotlib-based plots
# This requires ipympl and allows to capture Matplotlib plots as ipywidgets.
# %matplotlib widget

In [3]:
# set this to the host/port where the merlin data server is listening:
MERLIN_DATA_SOCKET = ('127.0.0.1', 6342)
SCAN_SIZE = (128, 128)

In [4]:
import logging

import numpy as np
import ipywidgets
from contextlib import contextmanager

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
# Sum all detector frames, result is a map of the detector
from libertem.udf.sum import SumUDF
# Sum up each detector frame, result is a bright field STEM image of the scan area
from libertem.udf.sumsigudf import SumSigUDF

# ImageGL-accelerated plot for fast live display
from libertem.viz.bqp import BQLive2DPlot
# Alternatively a version that uses the slower, but more mature Matplotlib
from libertem.viz.mpl import MPLLive2DPlot

INFO:empyre:Imported EMPyRe V-0.3.0 GIT-7531a074e8e81c3e02d65fad075edcd2c5408ad7


In [7]:
from libertem_live.api import LiveContext
from libertem_live.udf.monitor import SignalMonitorUDF

In [8]:
ctx = LiveContext()

### Camera setup routine

Different from offline processing, the shape, type and content of a dataset is not predetermined in live processing.
Instead, the data source has to be configured to supply the desired data. LiberTEM Live implements live datasets that behave similar to offline datasets. They include a setup routine that is called before each acquisition, and the actual data acquisition routines. The setup routine should be implemented by the user to configure the camera and the rest of the setup approriately to generate the desired data.

In order to also allow reliable de-initialization, this is not a regular function, but a context manager. See https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager for more
details. Once the setup is complete, the actual acquisition is started by entering the dataset's `start_acquisition()` context manager. At this point, the control is transferred to the dataset's
acquisition routines until the acquisition is complete.

The context manager that is defined here is supplied to the Context's `prepare_acquisition` method
in the following cell. `prepare_acquisition()` combines it with other parameters to create the complete
live dataset, i.e. a descriptor for the planned acquisition. `run_udf()` or `run_udf_iter()` enter this context
manager before processing starts on a live dataset and leave it after processing is finished.

In [9]:
@contextmanager
def medipix_setup(dataset, udfs):
    print("priming camera for acquisition")
    # TODO: medipix control socket commands go here
    # TODO interface to be tested, not supported in simulator yet

    # dataset.control.set('numframes', np.prod(SCAN_SIZE, dtype=np.int64))
    # dataset.control.set(...)

    # microscope.configure_scan()
    # microscope.start_scanning()
    print("running acquisition")
    with dataset.start_acquisition():
        yield
    print("camera teardown")
    # teardown routines go here

In [10]:
ds = ctx.prepare_acquisition(
    'merlin',
    medipix_setup,
    scan_size=SCAN_SIZE,
    host=MERLIN_DATA_SOCKET[0],
    port=MERLIN_DATA_SOCKET[1],
    control_port=None,  # deactivate control interface, not supported in simulator yet
    frames_per_partition=800,
    pool_size=2
)

In [11]:
udfs = [SumUDF(), SumSigUDF(), SignalMonitorUDF()]

In [12]:
LivePlot = BQLive2DPlot
# Uncomment below to use Matplotlib-based plotting
# See also the top of the notebook to select the correct matplotlib backend
# LivePlot = MPLLive2DPlot

p0 = LivePlot(ds, udfs[0])
p1 = LivePlot(ds, udfs[1])
p2 = LivePlot(ds, udfs[2])

In [13]:
outputs = []

for p in [p0, p1, p2]:
    # Capture the plots to display them in a grid later
    output = ipywidgets.Output()
    with output:
        p.display()
        # Some plot-specific tweaks for grid display
        if isinstance(p, BQLive2DPlot):
            p.figure.fig_margin={'top': 50, 'bottom': 0, 'left': 25, 'right': 25}
            p.figure.layout.width = '300px'
            p.figure.layout.height = '300px'
        elif isinstance(p, MPLLive2DPlot):
            p.fig.tight_layout()
            p.fig.set_size_inches((3, 3))
            p.fig.canvas.toolbar_position = 'bottom'
    outputs.append(output)

In [14]:
# Show the plot grid
ipywidgets.HBox(outputs)

### Sample output

The plots are not preserved when saving the notebook. They look like this:

![sample plot](run_on_merlin_data.png)

### Run one scan

The live plots above are updated with the results

In [15]:
%autoreload
ctx.run_udf(dataset=ds, udf=udfs, plots=[p0, p1, p2])

priming camera for acquisition
running acquisition
camera teardown


({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>})